In [1]:
%pip install catboost
import joblib
import json
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from itertools import combinations
from tensorflow.keras.losses import MeanSquaredError

Note: you may need to restart the kernel to use updated packages.


2025-05-21 21:01:32.388588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747861292.411024  249835 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747861292.418200  249835 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747861292.436820  249835 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747861292.436838  249835 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747861292.436840  249835 computation_placer.cc:177] computation placer alr

In [2]:
def evaluate_ensemble(y_true, y_pred, solver_name, target, model_combo, weights):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n[ENSEMBLE] Solver: {solver_name}, Target: {target}, Models: {model_combo}, Weights: {weights}")
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")
    return {
        "Solver": solver_name,
        "Target": target,
        "Models": "_".join(model_combo),
        "Weights": str(weights),
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    }

In [ ]:
def load_model_for_combination(model_name, solver_name, target, X_test, X_test_cnn):
    try:
        if model_name == "rf":
            path = f"./rf_reg/rf_models/rf_model_{solver_name}_{target}.joblib"
            if os.path.exists(path):
                model = joblib.load(path)
                return model_name, model.predict(X_test)
        elif model_name == "cb":
            path = f"./cb_reg/cb_models/cb_model_{solver_name}_{target}.cbm"
            if os.path.exists(path):
                model = CatBoostRegressor()
                model.load_model(path)
                return model_name, model.predict(X_test)
        elif model_name == "cnn":
            path = f"./cnn_reg/cnn_models/cnn_model_{solver_name}_{target}.h5"
            if os.path.exists(path):
                model = load_model(path, custom_objects={'mse': MeanSquaredError()})
                return model_name, model.predict(X_test_cnn).flatten()
        elif model_name == "mlp":
            path = f"./mlp_reg/mlp_models/mlp_model_{solver_name}_{target}.h5"
            if os.path.exists(path):
                model = load_model(path, custom_objects={'mse': MeanSquaredError()})
                return model_name, model.predict(X_test).flatten()
        elif model_name == "svm":
            path = f"./svm_reg/svm_models/svm_model_{solver_name}_{target}.joblib"
            if os.path.exists(path):
                model = joblib.load(path)
                return model_name, model.predict(X_test)
        elif model_name == "lr":
            path = f"./lr_reg/lr_models/lr_model_{solver_name}_{target}.joblib"
            if os.path.exists(path):
                model = joblib.load(path)
                return model_name, model.predict(X_test)
        elif model_name == "dt":
            path = f"./dt_reg/dt_models/dt_model_{solver_name}_{target}.joblib"
            if os.path.exists(path):
                model = joblib.load(path)
                return model_name, model.predict(X_test)
    except Exception as e:
        print(f"Failed to load {model_name} for {solver_name}, {target}: {e}")
    return None, None


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def rank_models_by_performance(solver_name, test_file, target, X_test, X_test_cnn, y_test):
    all_models = ["rf", "cb", "cnn", "mlp", "svm", "lr", "dt"]
    model_performance = {}

    for model_name in all_models:
        name, pred = load_model_for_combination(model_name, solver_name, target, X_test, X_test_cnn)
        if pred is not None:
            rmse = np.sqrt(mean_squared_error(y_test, pred))
            model_performance[model_name] = rmse
            print(f"{model_name} RMSE: {rmse:.4f}")
        else:
            print(f"{model_name} not evaluated.")

    # Sort models by RMSE
    ranked_models = sorted(model_performance, key=model_performance.get)
    print(f"Models ranked by RMSE for {target}: {ranked_models}")
    return ranked_models, model_performance

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os

def run_top_k_ensemble_dynamic(solver_name, train_file, test_file):
    
    df_train = pd.read_csv(train_file)
    df_test  = pd.read_csv(test_file)
    df_test.dropna(inplace=True)

 
    features = [
        "number_of_elements","capacity","max_weight","min_weight","mean_weight",
        "median_weight","std_weight","weight_range","max_profit","min_profit","mean_profit",
        "median_profit","std_profit","profit_range","renting_ratio","mean_weight_profit_ratio",
        "median_weight_profit_ratio","capacity_mean_weight_ratio","capacity_median_weight_ratio",
        "capacity_std_weight_ratio","std_weight_profit_ratio","weight_profit_correlation",
        "ram","cpu_cores"
    ]
    targets = ["solution_time","optimality_gap","peak_memory"]

   
    feat_scaler = StandardScaler().fit(df_train[features])
    X_test      = feat_scaler.transform(df_test[features])
    X_test_cnn  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

  
    y_scalers = {
        t: StandardScaler().fit(df_train[[t]].values)
        for t in targets
    }

    results = []

    for target in targets:
        y_test = df_test[target].values

       
        ranked_models, _ = rank_models_by_performance(
            solver_name, test_file, target, X_test, X_test_cnn, y_test
        )

        for k in [3, 5, 7]:
            ensemble_preds = []
            names = []

            for mname in ranked_models[:k]:
                name, pred = load_model_for_combination(
                    mname, solver_name, target, X_test, X_test_cnn
                )
                if pred is None:
                    continue

               
                if name in {"cnn","mlp","dt","rf","svm"}:
                    pred = y_scalers[target].inverse_transform(pred.reshape(-1,1)).flatten()

                ensemble_preds.append(pred)
                names.append(name)

            if not ensemble_preds:
                print(f"No models for Top-{k} on {solver_name}/{target}")
                continue

            #Equal‐weight average
            P = np.vstack(ensemble_preds)
            avg_pred = P.mean(axis=0)

            row = evaluate_ensemble(
                y_test, avg_pred, solver_name, target, tuple(names),
                weights=[1/len(P)]*len(P)
            )
            row["Top_K"] = k
            results.append(row)

    df_res = pd.DataFrame(results)
    out_path = "output.csv"
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df_res.to_csv(out_path, mode="a", index=False,
                  header=not os.path.exists(out_path))


In [ ]:
def run_ensembles_for_all_solvers(base_folder):
    for root, dirs, files in os.walk(base_folder):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            csv_files = os.listdir(folder_path)

            train_file = [f for f in csv_files if f.endswith("_train.csv")]
            test_file = [f for f in csv_files if f.endswith("_test.csv")]

            if test_file:
                train_fp = os.path.join(folder_path, train_file[0])
                test_fp = os.path.join(folder_path, test_file[0])
                solver_name = folder

                print(f"\nRunning for: {solver_name}")
                run_top_k_ensemble_dynamic(solver_name, train_fp, test_fp)

In [ ]:
base_folder = "./training_data"  #Path to training data folder
run_ensembles_for_all_solvers(base_folder)


Running ensembles for solver: or_min
rf RMSE: 2.0257
cb RMSE: 2.2013


I0000 00:00:1747861296.129000  249835 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22377 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:41:00.0, compute capability: 8.6
I0000 00:00:1747861296.131763  249835 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22377 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:61:00.0, compute capability: 8.6
I0000 00:00:1747861297.612938  250892 service.cc:152] XLA service 0x76d37c0067b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747861297.612981  250892 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
I0000 00:00:1747861297.612987  250892 service.cc:160]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2025-05-21 21:01:37.633006: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproduce

 1/18 ━━━━━━━━━━━━━━━━━━━━ 14s 859ms/step

I0000 00:00:1747861298.257769  250892 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step 
cnn RMSE: 2.1877


...

mlp RMSE: 4096.0000
svm RMSE: 4096.0000
lr RMSE: 0.0000
dt RMSE: 4096.0000
Top models ranked by RMSE for peak_memory: ['lr', 'rf', 'cnn', 'mlp', 'svm', 'dt']
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step

[ENSEMBLE] Solver: bb_min, Target: peak_memory, Models: ('lr', 'rf', 'cnn'), Weights: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
MAE: 0.0000, RMSE: 0.0000, R2: 1.0000


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step

[ENSEMBLE] Solver: bb_min, Target: peak_memory, Models: ('lr', 'rf', 'cnn', 'mlp', 'svm'), Weights: [0.2, 0.2, 0.2, 0.2, 0.2]


MAE: 0.0000, RMSE: 0.0000, R2: 1.0000
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step

[ENSEMBLE] Solver: bb_min, Target: peak_memory, Models: ('lr', 'rf', 'cnn', 'mlp', 'svm', 'dt'), Weights: [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
MAE: 0.0000, RMSE: 0.0000, R2: 1.0000
